<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Открытие-данных" data-toc-modified-id="Открытие-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Открытие данных</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Подготовка данных</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Проверка-лучшей-модели-на-тестовой-выборке" data-toc-modified-id="Проверка-лучшей-модели-на-тестовой-выборке-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Проверка лучшей модели на тестовой выборке</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Открытие данных

Импортируем библиотеки и открываем данные.

In [1]:
import pandas as pd 
import numpy as np 

from sklearn.pipeline import Pipeline

from tqdm.notebook import tqdm

import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.utils import shuffle

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import f1_score

import time
import warnings
warnings.filterwarnings('ignore')

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
df.tail()

,Unnamed: 0,text,toxic
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0
159291,159450,"""\nAnd ... I really don't think you understand...",0


In [6]:
df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Были импортированы библиотеки и открыты данные. Датасет с данными состоит из трёх столбцов - Unnamed: 0, test и toxic.
* `Unnamed: 0` - нужно удалить, так как он дублирует индексы строк.
* `test` - включает в себя текст твита.
* `toxic` - содержит булевы значения, которые показывают, является ли твит в столбце `test` токсичным или нет.

### Подготовка данных

Сначала удалим столбец `Unnamed: 0`.

In [7]:
tqdm.pandas()

In [8]:
df = df.drop(['Unnamed: 0'], axis=1)

Напишем функцию по очистке текста от нелатинских букв и замене этих букв на пробелы. Затем применим эту функцию.

In [9]:
def clear_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = ' '.join(text.split())
    return text

In [10]:
%%time

df['text'] = df['text'].apply(clear_text) 

CPU times: user 3.82 s, sys: 55 ms, total: 3.87 s
Wall time: 3.87 s


In [11]:
df.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i m se...,0
2,hey man i m really not trying to edit war it s...,0
3,more i can t make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0


In [12]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"Adj": wordnet.ADJ,               #прилагательное
                "N": wordnet.NOUN,                #существительное
                "V": wordnet.VERB,                #глагол
                "Adv": wordnet.ADV                #наречие
               }  
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

def lemm_text(text):
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    return ' '.join(text)

In [13]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [14]:
%%time

df['text'] = df['text'].progress_apply(lemm_text) 

  0%|          | 0/159292 [00:00<?, ?it/s]

CPU times: user 16min 53s, sys: 1min 40s, total: 18min 34s
Wall time: 18min 58s


In [15]:
df.head()

,text,toxic
0,explanation why the edits make under my userna...,0
1,d aww he match this background colour i m seem...,0
2,hey man i m really not try to edit war it s ju...,0
3,more i can t make any real suggestion on impro...,0
4,you sir be my hero any chance you remember wha...,0


In [16]:
#разделяю выборки в соотношении 90/10:
features = df.drop(['toxic'], axis = 1) 
target = df.toxic

features_train, features_test, target_train, target_test = train_test_split(features, target,
                                                                            test_size = 0.1, random_state = 12345)

In [18]:
print(features_train.shape,
      features_test.shape,
      target_train.shape,
      target_test.shape)

(143362, 1) (15930, 1) (143362,) (15930,)


Посмотрим на соотношение правильных значений к неправильным

In [19]:
indices_1 = [i for i, x in enumerate(target_train) if x == 1]
count_1 = len(indices_1)

indices_0 = [i for i, x in enumerate(target_train) if x == 0]
count_0 = len(indices_0)

print('Доля значений 1 в тренировочной выборке:', count_1 / (count_1 + count_0))

Доля значений 1 в тренировочной выборке: 0.1016587380198379


Видим, что правильных значений в выборке в разы меньше, чем неправильных. Проведём балансировку классов непосредственно в пайплане при обучении.

**Вывод**

Были подготовлены данные:
* проведена очистка данных
* проведена лемматизация данных
* были убраны стоп-слова
* произведён ресемплинг

## Обучение

Будем обучать три модели - логистическую регрессию, случайный лес и дерево решений.

### Логистическая регрессия

In [20]:
%%time

pipeline_lr = Pipeline([("vect", TfidfVectorizer(stop_words = 'english', sublinear_tf = True)), 
                        ("lr", LogisticRegression())])
    
parameters_lr = {'lr__solver': ('liblinear', 'saga', 'newton-cg', 'lbfgs'),
                 'lr__C': (.1, 1, 5, 10),
                 'lr__random_state': ([12345]),
                 'lr__max_iter': ([200]),
                 'lr__class_weight': (['balanced'])} 
                                                  
                                                  
gscv_lr = GridSearchCV(pipeline_lr, parameters_lr, scoring = 'f1', cv = 3, n_jobs = -1, verbose = 10)

gscv_lr.fit(features_train['text'], target_train)

mts = gscv_lr.cv_results_['mean_test_score']
lr_train_f1 = max(mts)

print('Показатель F1 для логистической регрессии равен', round(lr_train_f1, 2), 'при параметрах', gscv_lr.best_params_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV 1/3; 1/16] START lr__C=0.1, lr__class_weight=balanced, lr__max_iter=200, lr__random_state=12345, lr__solver=liblinear
[CV 1/3; 1/16] END lr__C=0.1, lr__class_weight=balanced, lr__max_iter=200, lr__random_state=12345, lr__solver=liblinear; total time=  11.0s
[CV 2/3; 1/16] START lr__C=0.1, lr__class_weight=balanced, lr__max_iter=200, lr__random_state=12345, lr__solver=liblinear
[CV 2/3; 1/16] END lr__C=0.1, lr__class_weight=balanced, lr__max_iter=200, lr__random_state=12345, lr__solver=liblinear; total time=  11.4s
[CV 3/3; 1/16] START lr__C=0.1, lr__class_weight=balanced, lr__max_iter=200, lr__random_state=12345, lr__solver=liblinear
[CV 3/3; 1/16] END lr__C=0.1, lr__class_weight=balanced, lr__max_iter=200, lr__random_state=12345, lr__solver=liblinear; total time=  11.4s
[CV 1/3; 2/16] START lr__C=0.1, lr__class_weight=balanced, lr__max_iter=200, lr__random_state=12345, lr__solver=saga
[CV 1/3; 2/16] END lr__C=0.1, lr__cl

### Случайный лес

In [21]:
%%time

pipeline_rfc = Pipeline([("vect", TfidfVectorizer(stop_words = 'english')), 
                         ("rfc", RandomForestClassifier())])
    
parameters_rfc = {'rfc__n_estimators': ([x for x in range(10, 20)]),
                  'rfc__random_state': ([12345]),
                  'rfc__max_depth': ([x for x in range(1, 10)]),
                  'rfc__criterion': (['entropy']),
                  'rfc__class_weight': (['balanced'])}

gscv_rfc = GridSearchCV(pipeline_rfc, parameters_rfc, scoring = 'f1', cv = 3, n_jobs = -1, verbose = 10)

gscv_rfc.fit(features_train['text'], target_train)

mts = gscv_rfc.cv_results_['mean_test_score']
rfc_train_f1 = max(mts)

print('Показатель F1 для случайного леса равен', round(rfc_train_f1, 2), 'при параметрах', gscv_rfc.best_params_)

Fitting 3 folds for each of 90 candidates, totalling 270 fits
[CV 1/3; 1/90] START rfc__class_weight=balanced, rfc__criterion=entropy, rfc__max_depth=1, rfc__n_estimators=10, rfc__random_state=12345
[CV 1/3; 1/90] END rfc__class_weight=balanced, rfc__criterion=entropy, rfc__max_depth=1, rfc__n_estimators=10, rfc__random_state=12345; total time=   6.0s
[CV 2/3; 1/90] START rfc__class_weight=balanced, rfc__criterion=entropy, rfc__max_depth=1, rfc__n_estimators=10, rfc__random_state=12345
[CV 2/3; 1/90] END rfc__class_weight=balanced, rfc__criterion=entropy, rfc__max_depth=1, rfc__n_estimators=10, rfc__random_state=12345; total time=   5.8s
[CV 3/3; 1/90] START rfc__class_weight=balanced, rfc__criterion=entropy, rfc__max_depth=1, rfc__n_estimators=10, rfc__random_state=12345
[CV 3/3; 1/90] END rfc__class_weight=balanced, rfc__criterion=entropy, rfc__max_depth=1, rfc__n_estimators=10, rfc__random_state=12345; total time=   5.5s
[CV 1/3; 2/90] START rfc__class_weight=balanced, rfc__criterio

### Дерево решений

In [22]:
%%time

pipeline_dtc = Pipeline([("vect", TfidfVectorizer(stop_words = 'english')), 
                         ("dtc", DecisionTreeClassifier())])
    
parameters_dtc = {'dtc__max_depth': ([x for x in range(1, 25)]),
                  'dtc__random_state': ([12345]), 
                  'dtc__class_weight': (['balanced'])}

gscv_dtc = GridSearchCV(pipeline_dtc, parameters_dtc, scoring = 'f1', cv = 3, n_jobs = -1, verbose = 10)

gscv_dtc.fit(features_train['text'], target_train)

mts = gscv_dtc.cv_results_['mean_test_score']
dtc_train_f1 = max(mts)

print('Показатель F1 для дерева решений равен', round(dtc_train_f1, 2), 'при параметрах', gscv_dtc.best_params_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV 1/3; 1/24] START dtc__class_weight=balanced, dtc__max_depth=1, dtc__random_state=12345
[CV 1/3; 1/24] END dtc__class_weight=balanced, dtc__max_depth=1, dtc__random_state=12345; total time=   7.0s
[CV 2/3; 1/24] START dtc__class_weight=balanced, dtc__max_depth=1, dtc__random_state=12345
[CV 2/3; 1/24] END dtc__class_weight=balanced, dtc__max_depth=1, dtc__random_state=12345; total time=   7.1s
[CV 3/3; 1/24] START dtc__class_weight=balanced, dtc__max_depth=1, dtc__random_state=12345
[CV 3/3; 1/24] END dtc__class_weight=balanced, dtc__max_depth=1, dtc__random_state=12345; total time=   7.5s
[CV 1/3; 2/24] START dtc__class_weight=balanced, dtc__max_depth=2, dtc__random_state=12345
[CV 1/3; 2/24] END dtc__class_weight=balanced, dtc__max_depth=2, dtc__random_state=12345; total time=   6.6s
[CV 2/3; 2/24] START dtc__class_weight=balanced, dtc__max_depth=2, dtc__random_state=12345
[CV 2/3; 2/24] END dtc__class_weight=balanced, d

Было обучено три модели - логистическая регрессия, случайный лес и дерево решений. Лучше всего себя показала модель логистической регрессии, показатель F1 для этой модели по результатам кросс-валидации равен 0.76.

### Проверка лучшей модели на тестовой выборке

In [23]:
predictions_test = gscv_lr.predict(features_test['text'])
lr_test_f1 = f1_score(target_test, predictions_test)
print('Финальный F1 для логистической регрессии равен', round(lr_test_f1, 2))

Финальный F1 для логистической регрессии равен 0.76


Данная модель имеет метрику качества F1 не меньше 0.75, всё хорошо.

## Выводы

* были загружены и подготовлены данные
* были обучены модели
* была выбрана наилучшая модель с метрикой F1 больше 0.75 - логистическая регрессия

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны